In [ ]:
# --- IMPORTS ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.metrics import r2_score, mean_squared_error
# Statsmodels & Sklearn Linear Models
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, callbacks, backend as K
from tensorflow.keras.optimizers import Adam
# Other Utilities
from sklearn.model_selection import ParameterGrid
import datetime
import warnings
import traceback
from collections import defaultdict
import os
import random # For setting seeds
import time

# --- WARNINGS CONFIGURATION ---
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="Mean of empty slice")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in log")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="Maximum number of iterations reached.*")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in scalar divide")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="Degrees of freedom <= 0 for slice")
pd.options.mode.chained_assignment = None

# --- TENSORFLOW/KERAS CONFIGURATION ---
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e: print(e)

# --------------------------------------------------------------------------
# FUNCTION DEFINITIONS
# --------------------------------------------------------------------------

# Step 1: Load and Prepare Dataset
def load_prepare_data(file_path):
    """ Loads, cleans, calculates returns/market cap, handles dates/IDs. """
    print(f"Laster data fra: {file_path}")
    try: df = pd.read_csv(file_path, low_memory=False)
    except FileNotFoundError: print(f"FEIL: Fil '{file_path}' ikke funnet."); return None
    print(f"Data lastet inn. Form: {df.shape}")
    date_col='Date'; id_col='Instrument'; price_col='ClosePrice'; shares_col='CommonSharesOutstanding'; rf_col='NorgesBank10Y'; sector_col='EconomicSector'
    if date_col not in df.columns: print("FEIL: Dato-kolonne mangler."); return None
    if id_col not in df.columns: print("FEIL: Instrument-ID mangler."); return None
    if price_col not in df.columns: print(f"FEIL: Pris-kolonne ('{price_col}') mangler."); return None
    df = df.rename(columns={id_col: 'Instrument'})
    df['Date'] = pd.to_datetime(df['Date']); df = df.sort_values(by=["Instrument", "Date"]).reset_index(drop=True); print("Dato konvertert og data sortert.")
    df["MonthlyReturn"] = df.groupby("Instrument")[price_col].pct_change(); df["MonthlyReturn"].fillna(0, inplace=True); df["MonthlyReturn"] = winsorize(df["MonthlyReturn"].values, limits=[0.01, 0.01]); print("Månedlig avkastning ('MonthlyReturn') beregnet/winsorisert.")
    if rf_col not in df.columns: df[rf_col] = 0; print(f"ADVARSEL: '{rf_col}' mangler, bruker 0.")
    df["MonthlyRiskFreeRate_t"] = df[rf_col] / 12 / 100 if df[rf_col].abs().max() > 1 else df[rf_col] / 12
    df["TargetReturn_t"] = df["MonthlyReturn"] - df["MonthlyRiskFreeRate_t"]
    print("Risikojustert avkastning ('TargetReturn_t') beregnet (modellens y).")
    df['NextMonthlyReturn_t+1'] = df.groupby('Instrument')['MonthlyReturn'].shift(-1); print("Neste måneds rå avkastning ('NextMonthlyReturn_t+1') beregnet.")
    if shares_col not in df.columns: print(f"FEIL: '{shares_col}' mangler for MarketCap."); return None
    df["MarketCap"] = df[price_col] * df[shares_col]; df['MarketCap_orig'] = df['MarketCap'].copy(); df['MarketCap'] = df['MarketCap'].fillna(0); print("Markedsverdi ('MarketCap') beregnet.")
    if sector_col in df.columns: df = pd.get_dummies(df, columns=[sector_col], prefix="Sector", dtype=int); print("Sektor dummy-variabler opprettet.")
    df.columns = df.columns.str.replace(" ", "_").str.replace("-", "_"); print("Kolonnenavn renset.")
    print("Log-transformerer variabler...")
    vars_to_log = ["MarketCap", "BM", "ClosePrice", "Volume", "CommonSharesOutstanding"]
    vars_to_log = [v if v in df.columns else v.lower() for v in vars_to_log]; vars_to_log = [v if v in df.columns else 'prc' if v == 'closeprice' else v for v in vars_to_log]; vars_to_log = [col for col in vars_to_log if col in df.columns]
    for var in vars_to_log:
        if var in df.columns:
             if pd.api.types.is_numeric_dtype(df[var]):
                 df[f"{var}_positive"] = df[var].where(df[var] > 1e-9, np.nan); df[f"log_{var}"] = np.log(df[f"{var}_positive"]); log_median = df[f"log_{var}"].median(); df[f"log_{var}"] = df[f"log_{var}"].fillna(log_median)
                 if pd.isna(log_median): df[f"log_{var}"] = df[f"log_{var}"].fillna(0)
                 df.drop(columns=[f"{var}_positive"], inplace=True)
             else: print(f"  ADVARSEL: Kolonne '{var}' ikke numerisk, hopper over log.")
    print("Log-transformasjon fullført.")
    if 'MarketCap_orig' not in df.columns and 'MarketCap' in df.columns: df['MarketCap_orig'] = df['MarketCap'].copy()
    return df

# Step 1.5: Rank Standardization
def rank_standardize_features(df, features_to_standardize):
    """ Performs rank standardization cross-sectionally to [-1, 1]. """
    print(f"Rank standardiserer {len(features_to_standardize)} features...")
    if 'Date' not in df.columns: print("FEIL: 'Date' mangler."); return df
    features_present = [f for f in features_to_standardize if f in df.columns]
    if len(features_present) < len(features_to_standardize): print(f"  ADVARSEL: Manglet features for standardisering: {[f for f in features_to_standardize if f not in features_present]}")
    if not features_present: print("  Ingen features å standardisere."); return df
    def rank_transform(x): x_numeric = pd.to_numeric(x, errors='coerce'); ranks = x_numeric.rank(pct=True); return ranks * 2 - 1
    try:
        ranked_cols = df.groupby('Date')[features_present].transform(rank_transform)
        df[features_present] = ranked_cols
    except Exception as e:
        print(f"  ADVARSEL rank standardisering (transform): {e}. Prøver apply.")
        try:
            df_std = df.set_index('Date')
            for col in features_present: df_std[col] = df_std.groupby(level=0)[col].apply(rank_transform)
            df = df_std.reset_index()
        except Exception as e2: print(f"  FEIL: Også alternativ standardisering feilet: {e2}"); return df
    print("Rank standardisering fullført.")
    return df

# Step 2: Define Feature Sets
def define_features(df):
    """ Defines feature sets: all numeric and specific OLS-3 features. """
    print("Identifiserer numeriske features...")
    if df is None or df.empty: print("  FEIL: DataFrame er tom."); return [], []
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist(); print(f"  Funnet {len(numeric_cols)} numeriske kolonner totalt.")
    cols_to_exclude = ['Instrument', 'Date', 'level_0', 'index', 'Year', 'MonthYear', 'TargetReturn_t', 'NextMonthlyReturn_t+1', 'MonthlyReturn', 'MonthlyRiskFreeRate_t', 'MarketCap_orig', 'rank', 'DecileRank', 'eq_weights', 'me_weights']
    cols_to_exclude.extend([col for col in df.columns if 'return_stock' in col or 'return_portfolio' in col])
    log_cols = [col for col in numeric_cols if col.startswith('log_')]; originals_of_log = [col.replace('log_','') for col in log_cols]
    common_originals = ['MarketCap', 'ClosePrice', 'prc', 'Volume', 'CommonSharesOutstanding', 'BM']
    originals_to_exclude = [orig for orig in originals_of_log if orig in df.columns and orig in common_originals]; cols_to_exclude.extend(originals_to_exclude)
    cols_to_exclude = list(set(cols_to_exclude)); potential_features = [col for col in numeric_cols if col not in cols_to_exclude]
    all_numeric_features = []
    for col in potential_features:
        if col in df.columns:
            if df[col].nunique(dropna=True) > 1 and df[col].std(ddof=0, skipna=True) > 1e-9: all_numeric_features.append(col)
    all_numeric_features = sorted(list(set(all_numeric_features))); print(f"  Identifisert {len(all_numeric_features)} potensielle numeriske features totalt.")
    ols3_features_def = ["log_BM", "Momentum_12M", "log_MarketCap"]
    ols3_features = [f for f in ols3_features_def if f in all_numeric_features]
    missing_ols3 = [f for f in ols3_features_def if f not in ols3_features]
    if missing_ols3: print(f"  ADVARSEL: Følgende OLS3 features mangler: {missing_ols3}")
    if not ols3_features: print("  ADVARSEL: Ingen OLS3 features funnet.")
    else: print(f"  Identifisert {len(ols3_features)} OLS3 features: {ols3_features}")
    return all_numeric_features, ols3_features

# Step 3: Handle Missing / Infinite Values
def clean_data(df, numeric_features_to_impute, essential_cols_for_dropna, target="TargetReturn_t"):
    """ Handles inf/NaN. Fills numeric with median. Drops rows with NaN in essential cols. Filters MarketCap_orig <= 0. """
    print("Starter datarensing (missing/inf)...")
    initial_rows = len(df); features_present = [f for f in numeric_features_to_impute if f in df.columns];
    if not features_present: print("  Ingen features for imputering.");
    else:
        non_numeric = [f for f in features_present if not pd.api.types.is_numeric_dtype(df[f])]
        if non_numeric: print(f"  ADVARSEL: Ikke-numeriske i impute-liste: {non_numeric}. Ignoreres."); features_present = [f for f in features_present if f not in non_numeric]
        if features_present:
            inf_mask = df[features_present].isin([np.inf, -np.inf])
            if inf_mask.any().any(): print(f"  Erstatter inf med NaN..."); df[features_present] = df[features_present].replace([np.inf, -np.inf], np.nan)
            nan_counts_before = df[features_present].isnull().sum(); medians = df[features_present].median(skipna=True); df[features_present] = df[features_present].fillna(medians); nan_counts_after = df[features_present].isnull().sum()
            imputed_cols = (nan_counts_before - nan_counts_after); print(f"  NaNs imputert med median i {imputed_cols[imputed_cols > 0].count()} feature kolonner.")
            if medians.isnull().any(): cols_nan_median = medians[medians.isnull()].index.tolist(); print(f"  ADVARSEL: Median NaN for: {cols_nan_median}. Fyller med 0."); df[cols_nan_median] = df[cols_nan_median].fillna(0)
    essential_cols_present = [col for col in essential_cols_for_dropna if col in df.columns]
    for col in [target,'NextMonthlyReturn_t+1','MarketCap_orig']:
         if col in df.columns and col not in essential_cols_present: essential_cols_present.append(col)
    unique_essential_cols = sorted(list(set(essential_cols_present)))
    if unique_essential_cols:
        rows_before_dropna = len(df); df = df.dropna(subset=unique_essential_cols); rows_dropped = rows_before_dropna - len(df)
        if rows_dropped > 0: print(f"  Fjernet {rows_dropped} rader pga. NaN i essensielle kolonner: {unique_essential_cols}")
        else: print(f"  Ingen rader fjernet pga NaN i essensielle kolonner.")
    mc_orig_col = 'MarketCap_orig'
    if mc_orig_col in df.columns:
        rows_before_mc_filter = len(df); df = df[df[mc_orig_col] > 0]; rows_dropped_mc = rows_before_mc_filter - len(df)
        if rows_dropped_mc > 0: print(f"  Fjernet {rows_dropped_mc} rader der {mc_orig_col} <= 0.")
        else: print(f"  Ingen rader fjernet pga {mc_orig_col} <=0.")
    else: print(f"  Advarsel: '{mc_orig_col}' ikke funnet for filtrering.")
    final_rows = len(df); print(f"Datarensing fullført. Form: {df.shape}. Fjernet totalt {initial_rows - final_rows} rader.")
    if df.empty: print("FEIL: Ingen data igjen etter rensing.")
    return df

# Step 4: Yearly Rolling Window Splits
def get_yearly_rolling_splits(df, initial_train_years, val_years, test_years=1):
    """ Creates yearly rolling window splits with expanding training and rolling validation. """
    # ... (Function code is identical to previous correct versions) ...
    if "Date" not in df.columns: raise ValueError("'Date'-kolonnen mangler i DataFrame.")
    df['Year'] = df["Date"].dt.year; unique_years = sorted(df["Year"].unique()); n_unique_years = len(unique_years)
    print(f"Unike år i data: {n_unique_years} ({unique_years[0]} - {unique_years[-1]})")
    if n_unique_years < initial_train_years + val_years + test_years: df.drop(columns=['Year'], inplace=True, errors='ignore'); raise ValueError(f"Ikke nok unike år ({n_unique_years}) for vindu InitTrain={initial_train_years}, Val={val_years}, Test={test_years}.")
    first_test_year_index = initial_train_years + val_years
    if first_test_year_index >= n_unique_years: df.drop(columns=['Year'], inplace=True, errors='ignore'); raise ValueError("InitTrain+Val er for lang for datasettet.")
    first_test_year = unique_years[first_test_year_index]; last_test_year = unique_years[-test_years]
    num_windows = last_test_year - first_test_year + 1
    if num_windows <= 0: df.drop(columns=['Year'], inplace=True, errors='ignore'); raise ValueError("Beregnet antall vinduer er null/negativt.")
    print(f"Genererer {num_windows} årlige rullerende vinduer..."); print(f"(Første testår: {first_test_year}, Siste testår: {last_test_year})")
    for i in range(num_windows):
        current_test_start_year=first_test_year+i; current_test_end_year=current_test_start_year+test_years-1; current_val_end_year=current_test_start_year-1; current_val_start_year=current_val_end_year-val_years+1; current_train_end_year=current_val_start_year-1; current_train_start_year=unique_years[0]
        train_idx = df[(df['Year'] >= current_train_start_year) & (df['Year'] <= current_train_end_year)].index
        val_idx = df[(df['Year'] >= current_val_start_year) & (df['Year'] <= current_val_end_year)].index
        test_idx = df[(df['Year'] >= current_test_start_year) & (df['Year'] <= current_test_end_year)].index
        train_dates = df.loc[train_idx, "Date"].agg(['min', 'max']) if not train_idx.empty else None
        val_dates = df.loc[val_idx, "Date"].agg(['min', 'max']) if not val_idx.empty else None
        test_dates = df.loc[test_idx, "Date"].agg(['min', 'max']) if not test_idx.empty else None
        print(f"\n  Vindu {i+1}/{num_windows}:"); print(f"    Train: {current_train_start_year}-{current_train_end_year} ({len(train_idx)} obs)"); print(f"    Val  : {current_val_start_year}-{current_val_end_year} ({len(val_idx)} obs)"); print(f"    Test : {current_test_start_year}-{current_test_end_year} ({len(test_idx)} obs)")
        yield train_idx, val_idx, test_idx, train_dates, val_dates, test_dates
    if 'Year' in df.columns: df.drop(columns=['Year'], inplace=True, errors='ignore')

# Step 5a: Run Statsmodels Model on Window (Only OLS-3+H)
def run_statsmodels_on_window(X_train_scaled, y_train, X_test_scaled, y_test, method='OLS-3+H'):
    """ Trains and evaluates OLS-3+H (RLM Huber) for one window. Expects scaled data. """
    # ... (Function code is identical to previous correct versions) ...
    X_train_scaled_const = sm.add_constant(X_train_scaled, has_constant='add'); X_test_scaled_const = sm.add_constant(X_test_scaled, has_constant='add')
    model = None; preds_oos = np.full(y_test.shape, np.nan); preds_is = np.full(y_train.shape, np.nan); r2_oos, mse_oos, sharpe_oos, r2_is, sharpe_is = (np.nan,) * 5
    method_upper = method.upper()
    try:
        if method_upper == 'OLS-3+H': model = sm.RLM(y_train, X_train_scaled_const, M=sm.robust.norms.HuberT()).fit()
        else: raise ValueError(f"Ugyldig Statsmodels metode: {method}. Kun 'OLS-3+H' støttes her.")
        if X_test_scaled.shape[0] > 0:
            preds_oos = model.predict(X_test_scaled_const); nan_preds_oos = ~np.isfinite(preds_oos); preds_oos[nan_preds_oos] = 0
            y_test_valid = y_test[~nan_preds_oos]; preds_oos_valid = preds_oos[~nan_preds_oos]
            if len(preds_oos_valid) > 1: ss_res_oos = np.sum((y_test_valid - preds_oos_valid)**2); ss_tot_oos = np.sum(y_test_valid**2); r2_oos = 1 - (ss_res_oos / ss_tot_oos) if ss_tot_oos > 1e-9 else np.nan; mse_oos = mean_squared_error(y_test_valid, preds_oos_valid); pred_std_oos = np.std(preds_oos_valid); sharpe_oos = (np.mean(preds_oos_valid) / pred_std_oos)*np.sqrt(12) if pred_std_oos > 1e-9 else np.nan
        preds_is = model.predict(X_train_scaled_const); nan_preds_is = ~np.isfinite(preds_is); preds_is[nan_preds_is] = 0
        y_train_valid = y_train[~nan_preds_is]; preds_is_valid = preds_is[~nan_preds_is]
        if len(preds_is_valid) > 1: ss_res_is = np.sum((y_train_valid - preds_is_valid)**2); ss_tot_is = np.sum(y_train_valid**2); r2_is = 1 - (ss_res_is / ss_tot_is) if ss_tot_is > 1e-9 else np.nan; pred_std_is = np.std(preds_is_valid); sharpe_is = (np.mean(preds_is_valid) / pred_std_is)*np.sqrt(12) if pred_std_is > 1e-9 else np.nan
    except Exception as e: print(f"  FEIL {method_upper}: {e}"); model = None
    return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is, sharpe_is, model

# Step 5b: Run Sklearn Model on Window
def run_sklearn_on_window(X_train_scaled, y_train, X_test_scaled, y_test, model_type='OLS', n_components=10, alphas_ridge=None, alphas_lasso=None, l1_ratio_elastic=0.5):
    """ Trains and evaluates sklearn models. Expects scaled data. """
    # ... (Function code is identical to previous correct versions) ...
    model = None; preds_oos = np.full(y_test.shape, np.nan); preds_is = np.full(y_train.shape, np.nan); r2_oos, mse_oos, sharpe_oos, r2_is, sharpe_is = (np.nan,) * 5
    model_name = model_type.upper(); SEED=42
    if alphas_ridge is None: alphas_ridge = np.logspace(-4, 4, 10)
    if alphas_lasso is None: alphas_lasso = np.logspace(-6, -1, 10)
    try:
        if model_name == 'OLS': model = LinearRegression(fit_intercept=True)
        elif model_name == 'PLS': actual_n = min(n_components, X_train_scaled.shape[0], X_train_scaled.shape[1]); actual_n = max(1, actual_n); model = PLSRegression(n_components=actual_n, scale=False)
        elif model_name == 'PCR': actual_n = min(n_components, X_train_scaled.shape[0], X_train_scaled.shape[1]); actual_n = max(1, actual_n); model = Pipeline([('pca', PCA(n_components=actual_n)), ('linear_regression', LinearRegression(fit_intercept=True))])
        elif model_name == 'RIDGE': model = RidgeCV(alphas=alphas_ridge, fit_intercept=True, scoring='neg_mean_squared_error')
        elif model_name == 'LASSO': model = LassoCV(alphas=alphas_lasso, fit_intercept=True, cv=5, max_iter=1000, tol=0.001, random_state=SEED, n_jobs=-1)
        elif model_name == 'ELASTICNET': model = ElasticNetCV(alphas=alphas_lasso, l1_ratio=[l1_ratio_elastic], fit_intercept=True, cv=5, max_iter=1000, tol=0.001, random_state=SEED, n_jobs=-1)
        else: raise ValueError(f"Ugyldig Scikit-learn model type: {model_type}")
        model.fit(X_train_scaled, y_train)
        if X_test_scaled.shape[0] > 0:
            preds_oos = model.predict(X_test_scaled); nan_preds_oos = ~np.isfinite(preds_oos); preds_oos[nan_preds_oos] = 0
            if preds_oos.ndim > 1: preds_oos = preds_oos.flatten()
            y_test_valid = y_test[~nan_preds_oos]; preds_oos_valid = preds_oos[~nan_preds_oos]
            if len(preds_oos_valid) > 1: ss_res_oos = np.sum((y_test_valid - preds_oos_valid)**2); ss_tot_oos = np.sum(y_test_valid**2); r2_oos = 1 - (ss_res_oos / ss_tot_oos) if ss_tot_oos > 1e-9 else np.nan; mse_oos = mean_squared_error(y_test_valid, preds_oos_valid); pred_std_oos = np.std(preds_oos_valid); sharpe_oos = (np.mean(preds_oos_valid) / pred_std_oos)*np.sqrt(12) if pred_std_oos > 1e-9 else np.nan
        preds_is = model.predict(X_train_scaled); nan_preds_is = ~np.isfinite(preds_is); preds_is[nan_preds_is] = 0
        if preds_is.ndim > 1: preds_is = preds_is.flatten()
        y_train_valid = y_train[~nan_preds_is]; preds_is_valid = preds_is[~nan_preds_is]
        if len(preds_is_valid) > 1: ss_res_is = np.sum((y_train_valid - preds_is_valid)**2); ss_tot_is = np.sum(y_train_valid**2); r2_is = 1 - (ss_res_is / ss_tot_is) if ss_tot_is > 1e-9 else np.nan; pred_std_is = np.std(preds_is_valid); sharpe_is = (np.mean(preds_is_valid) / pred_std_is)*np.sqrt(12) if pred_std_is > 1e-9 else np.nan
    except Exception as e: print(f"  FEIL {model_name}: {e}"); model = None
    return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is, sharpe_is, model

# Step 5c: Run Neural Network Model on Window
def build_nn_model(input_shape, nn_config, lambda1):
    """Builds a Keras Sequential NN model based on the config."""
    # ... (Function code is identical to previous correct versions) ...
    model = keras.Sequential(name=nn_config['name']); model.add(layers.Input(shape=(input_shape,)))
    for units in nn_config['hidden_units']: model.add(layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l1(lambda1)))
    model.add(layers.Dense(1, activation='linear')); return model

def run_nn_on_window(X_train, y_train, X_val, y_val, X_test, y_test, nn_config, param_grid, epochs=100, batch_size=10000, patience=5, ensemble_size=10):
    """ Trains an ensembled NN with tuning for a SINGLE YEARLY WINDOW. """
    # ... (Function code is identical to previous correct versions, returns optimal params) ...
    model_name = nn_config['name']; optim_param_found = None; optimal_lambda1 = np.nan; optimal_lr = np.nan; best_val_mse = np.inf
    input_shape = X_train.shape[1]; preds_oos = np.full(y_test.shape[0], np.nan); r2_oos, mse_oos, sharpe_oos, r2_is_train_val = (np.nan,) * 4
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=0); terminate_nan = callbacks.TerminateOnNaN()
    if X_val.shape[0] < 2: print(f"    ADVARSEL ({model_name}): Val set too small. Skipping."); return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is_train_val, optimal_lambda1, optimal_lr
    for params in param_grid:
        lambda1 = params['lambda1']; learning_rate = params['learning_rate']; val_preds_ensemble = []
        try:
            for ens_idx in range(ensemble_size):
                K.clear_session(); tf.random.set_seed(SEED + ens_idx)
                model_val = build_nn_model(input_shape, nn_config, lambda1); optimizer = Adam(learning_rate=learning_rate); model_val.compile(optimizer=optimizer, loss='mean_squared_error')
                history = model_val.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, terminate_nan], verbose=0)
                if not np.isnan(history.history['val_loss']).any(): val_preds_ensemble.append(model_val.predict(X_val, batch_size=batch_size).flatten())
                else: val_preds_ensemble = []; break
            if not val_preds_ensemble: continue
            avg_val_preds = np.mean(np.array(val_preds_ensemble), axis=0); current_mse_val = mean_squared_error(y_val[np.isfinite(avg_val_preds)], avg_val_preds[np.isfinite(avg_val_preds)])
            if not np.isnan(current_mse_val) and current_mse_val < best_val_mse: best_val_mse = current_mse_val; optim_param_found = params
        except Exception as e: continue
    if optim_param_found is None: print(f"    FEIL ({model_name}): Tuning feilet."); return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is_train_val, optimal_lambda1, optimal_lr
    optimal_lambda1 = optim_param_found['lambda1']; optimal_lr = optim_param_found['learning_rate']
    X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val)); test_preds_ensemble = []; is_preds_ensemble = []
    try:
        for i in range(ensemble_size):
            K.clear_session(); tf.random.set_seed(SEED + i + ensemble_size)
            model_final = build_nn_model(input_shape, nn_config, optimal_lambda1); optimizer = Adam(learning_rate=optimal_lr); model_final.compile(optimizer=optimizer, loss='mean_squared_error')
            history_final = model_final.fit(X_train_val, y_train_val, epochs=epochs, batch_size=batch_size, callbacks=[terminate_nan], verbose=0)
            if not np.isnan(history_final.history['loss']).any():
                 if X_test.shape[0] > 0: test_preds_ensemble.append(model_final.predict(X_test, batch_size=batch_size).flatten())
                 is_preds_ensemble.append(model_final.predict(X_train_val, batch_size=batch_size).flatten())
            else: test_preds_ensemble = []; is_preds_ensemble = []; break
        if X_test.shape[0] > 0 and not test_preds_ensemble: raise ValueError(f"Final OOS ensemble failed ({model_name}).")
        if not is_preds_ensemble: raise ValueError(f"Final IS ensemble failed ({model_name}).")
        if X_test.shape[0] > 0: preds_oos = np.mean(np.array(test_preds_ensemble), axis=0)
        preds_is = np.mean(np.array(is_preds_ensemble), axis=0)
    except Exception as e: print(f"  FEIL final {model_name} fit/predict: {e}"); preds_oos.fill(np.nan); return preds_oos, np.nan, np.nan, np.nan, np.nan, optimal_lambda1, optimal_lr
    if X_test.shape[0] > 0:
        nan_preds_oos_mask = ~np.isfinite(preds_oos); preds_oos[nan_preds_oos_mask] = 0
        valid_oos_mask = np.isfinite(y_test) & np.isfinite(preds_oos); y_test_valid = y_test[valid_oos_mask]; preds_oos_valid = preds_oos[valid_oos_mask]
        if len(preds_oos_valid) > 1: ss_res_oos = np.sum((y_test_valid - preds_oos_valid)**2); ss_tot_oos = np.sum(y_test_valid**2); r2_oos = 1 - (ss_res_oos / ss_tot_oos) if ss_tot_oos > 1e-9 else np.nan; mse_oos = mean_squared_error(y_test_valid, preds_oos_valid); pred_std_oos = np.std(preds_oos_valid); sharpe_oos = (np.mean(preds_oos_valid)/pred_std_oos)*np.sqrt(12) if pred_std_oos > 1e-9 else np.nan
    nan_preds_is_mask = ~np.isfinite(preds_is); preds_is[nan_preds_is_mask] = 0
    valid_is_mask = np.isfinite(y_train_val) & np.isfinite(preds_is); y_train_val_valid = y_train_val[valid_is_mask]; preds_is_valid = preds_is[valid_is_mask]
    if len(preds_is_valid) > 1: ss_res_is = np.sum((y_train_val_valid - preds_is_valid)**2); ss_tot_is = np.sum(y_train_val_valid**2); r2_is_train_val = 1 - (ss_res_is / ss_tot_is) if ss_tot_is > 1e-9 else np.nan
    return preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is_train_val, optimal_lambda1, optimal_lr


# Step 6: Calculate Portfolio Performance
def calculate_portfolio_performance(results_df, original_df, prediction_cols, risk_free_rate_col='NorgesBank10Y', filter_small_caps=False):
    """ Calculates decile portfolio performance for a list of prediction columns. """
    # ... (Function code is identical to previous correct versions) ...
    print("\n--- Starter generalisert porteføljekonstruksjon og analyse ---")
    if filter_small_caps: print(">>> Filtrering av 10% minste selskaper per måned er AKTIVERT <<<")
    else: print(">>> Filtrering av 10% minste selskaper per måned er DEAKTIVERT <<<")
    required_original_cols = ['Date', 'Instrument', 'MarketCap_orig', 'NextMonthlyReturn_t+1', risk_free_rate_col, 'MonthlyRiskFreeRate_t']
    if not all(col in original_df.columns for col in required_original_cols): print(f"FEIL: Mangler kolonner i original_df: {[col for col in required_original_cols if col not in original_df.columns]}"); return None, None
    required_results_cols = ['Date', 'Instrument', 'TargetReturn_t'] + prediction_cols
    if not all(col in results_df.columns for col in required_results_cols): print(f"FEIL: Mangler kolonner i results_df: {[col for col in required_results_cols if col not in results_df.columns]}"); return None, None
    portfolio_data = pd.merge(results_df, original_df[['Date', 'Instrument', 'MarketCap_orig', 'NextMonthlyReturn_t+1', risk_free_rate_col, 'MonthlyRiskFreeRate_t']], on=['Date', 'Instrument'], how='inner')
    portfolio_data.rename(columns={'MarketCap_orig': 'me'}, inplace=True)
    print(f"Data for porteføljeanalyse etter merge: {portfolio_data.shape}")
    portfolio_data['MonthYear'] = portfolio_data['Date'].dt.to_period('M')
    monthly_rf_map = portfolio_data.groupby('MonthYear')['MonthlyRiskFreeRate_t'].mean().shift(-1)
    portfolio_data['NextMonthRiskFreeRate_t+1'] = portfolio_data['MonthYear'].map(monthly_rf_map)
    critical_cols = prediction_cols + ['TargetReturn_t', 'NextMonthlyReturn_t+1', 'me', 'NextMonthRiskFreeRate_t+1']
    rows_before_crit_dropna = len(portfolio_data); portfolio_data = portfolio_data.dropna(subset=critical_cols)
    print(f"Størrelse etter dropna på kritiske kolonner: {portfolio_data.shape} (fjernet {rows_before_crit_dropna - len(portfolio_data)} rader)")
    rows_before_mc_filter = len(portfolio_data); portfolio_data = portfolio_data[portfolio_data['me'] > 0]
    print(f"Størrelse etter sikring av me > 0: {portfolio_data.shape} (fjernet {rows_before_mc_filter - len(portfolio_data)} rader)")
    if portfolio_data.empty: print("Ingen data igjen for porteføljekonstruksjon."); return None, None
    portfolio_data['NextMonthExcessReturn_t+1'] = portfolio_data['NextMonthlyReturn_t+1'] - portfolio_data['NextMonthRiskFreeRate_t+1']
    print("Beregnet neste måneds EXCESS avkastning ('NextMonthExcessReturn_t+1').")

    all_monthly_results = []; hl_monthly_dfs_plotting = {}; hl_sharpe_ratios = defaultdict(lambda: {'hl_sharpe_ew': np.nan, 'hl_sharpe_vw': np.nan})
    unique_months = sorted(portfolio_data['MonthYear'].unique()); print(f"Itererer gjennom {len(unique_months)} måneder for porteføljer...")
    for month in unique_months:
        monthly_data_full = portfolio_data[portfolio_data['MonthYear'] == month].copy()
        if filter_small_caps:
            if 'me' in monthly_data_full.columns and len(monthly_data_full) > 10: monthly_data_filtered = monthly_data_full[monthly_data_full['me'] >= monthly_data_full['me'].quantile(0.10)].copy()
            else: monthly_data_filtered = monthly_data_full.copy()
        else: monthly_data_filtered = monthly_data_full.copy()
        if len(monthly_data_filtered) < 10: continue
        for model_pred_col in prediction_cols:
            model_label = model_pred_col[len('yhat_'):].upper() if model_pred_col.startswith('yhat_') else model_pred_col.upper()
            if model_label.lower() == 'ols-3': model_label = 'OLS-3' # Correct label reconstruction
            elif model_label.lower() == 'ols-3h': model_label = 'OLS-3+H' # Correct label reconstruction
            monthly_data = monthly_data_filtered.copy().sort_values(model_pred_col)
            try:
                monthly_data['Rank'] = monthly_data[model_pred_col].rank(method='first')
                monthly_data['Decile'] = pd.qcut(monthly_data['Rank'], 10, labels=False, duplicates='drop')
                if monthly_data['Decile'].nunique() < 10: continue
            except ValueError: continue
            monthly_data.drop(columns=['Rank'], inplace=True)
            monthly_data['ew_weights'] = 1 / monthly_data.groupby('Decile')['Instrument'].transform('size')
            market_cap_sum = monthly_data.groupby('Decile')['me'].transform('sum')
            monthly_data['vw_weights'] = np.where(market_cap_sum > 0, monthly_data['me'] / market_cap_sum, 0)
            monthly_data['ew_raw_ret_t+1'] = monthly_data['NextMonthlyReturn_t+1'] * monthly_data['ew_weights']
            monthly_data['vw_raw_ret_t+1'] = monthly_data['NextMonthlyReturn_t+1'] * monthly_data['vw_weights']
            monthly_data['ew_excess_ret_t+1'] = monthly_data['NextMonthExcessReturn_t+1'] * monthly_data['ew_weights']
            monthly_data['vw_excess_ret_t+1'] = monthly_data['NextMonthExcessReturn_t+1'] * monthly_data['vw_weights']
            monthly_data['ew_pred_ret_t'] = monthly_data[model_pred_col] * monthly_data['ew_weights']
            monthly_data['vw_pred_ret_t'] = monthly_data[model_pred_col] * monthly_data['vw_weights']
            monthly_portfolio_returns = monthly_data.groupby('Decile').agg(
                ew_ret_raw=('ew_raw_ret_t+1', 'sum'), vw_ret_raw=('vw_raw_ret_t+1', 'sum'),
                ew_ret_excess=('ew_excess_ret_t+1', 'sum'), vw_ret_excess=('vw_excess_ret_t+1', 'sum'),
                ew_ret_pred=('ew_pred_ret_t', 'sum'), vw_ret_pred=('vw_pred_ret_t', 'sum')
            ).reset_index()
            monthly_portfolio_returns['MonthYear'] = month; monthly_portfolio_returns['Model'] = model_label
            all_monthly_results.append(monthly_portfolio_returns)

    if not all_monthly_results: print("Ingen månedlige porteføljeresultater generert."); return None, None
    combined_results_df = pd.concat(all_monthly_results).reset_index(drop=True)
    print(f"\nSamlet {len(combined_results_df)} månedlige desilresultater.")
    performance_summary_list = []; model_names_in_results = combined_results_df['Model'].unique()
    print(f"Analyserer ytelse for modeller: {list(model_names_in_results)}")
    for model_name in model_names_in_results:
        model_results = combined_results_df[combined_results_df['Model'] == model_name].copy();
        if model_results.empty: continue
        decile_performance = model_results.groupby('Decile').agg(
            ew_pred_mean=('ew_ret_pred', 'mean'), vw_pred_mean=('vw_ret_pred', 'mean'),
            ew_excess_mean=('ew_ret_excess', 'mean'), vw_excess_mean=('vw_ret_excess', 'mean'),
            ew_raw_std=('ew_ret_raw', 'std'), vw_raw_std=('vw_ret_raw', 'std'),
            n_months=('MonthYear', 'count')
        ).reset_index()
        annualization_factor = 12
        decile_performance['ew_sharpe'] = np.where(pd.notna(decile_performance['ew_raw_std']) & (decile_performance['ew_raw_std'] > 1e-9), (decile_performance['ew_excess_mean'] / decile_performance['ew_raw_std']) * np.sqrt(annualization_factor), np.nan)
        decile_performance['vw_sharpe'] = np.where(pd.notna(decile_performance['vw_raw_std']) & (decile_performance['vw_raw_std'] > 1e-9), (decile_performance['vw_excess_mean'] / decile_performance['vw_raw_std']) * np.sqrt(annualization_factor), np.nan)
        hl_monthly_df = pd.DataFrame(); hl_stats_df = pd.DataFrame(); ew_sharpe_hl_val = np.nan; vw_sharpe_hl_val = np.nan
        if 0 in model_results['Decile'].values and 9 in model_results['Decile'].values:
            metrics_to_pivot = ['ew_ret_raw', 'vw_ret_raw', 'ew_ret_excess', 'vw_ret_excess', 'ew_ret_pred', 'vw_ret_pred']
            hl_monthly_calc = {}
            for metric in metrics_to_pivot:
                pivoted = model_results.pivot(index='MonthYear', columns='Decile', values=metric)
                col_name_hl = metric + '_HL'
                if 0 in pivoted.columns and 9 in pivoted.columns:
                    valid_rows = pivoted[9].notna() & pivoted[0].notna()
                    hl_monthly_calc[col_name_hl] = pd.Series(np.nan, index=pivoted.index); hl_monthly_calc[col_name_hl][valid_rows] = pivoted.loc[valid_rows, 9] - pivoted.loc[valid_rows, 0]
                else: hl_monthly_calc[col_name_hl] = pd.Series(np.nan, index=pivoted.index)
            hl_monthly_df = pd.DataFrame(hl_monthly_calc).reset_index()
            hl_monthly_dfs_plotting[model_name] = hl_monthly_df.copy()
            essential_hl_cols = ['ew_ret_excess_HL', 'ew_ret_raw_HL', 'vw_ret_excess_HL', 'vw_ret_raw_HL']
            cols_exist = [col for col in essential_hl_cols if col in hl_monthly_df.columns]
            if cols_exist: hl_monthly_df_clean = hl_monthly_df.dropna(subset=cols_exist).copy()
            else: hl_monthly_df_clean = hl_monthly_df.copy()
            if not hl_monthly_df_clean.empty:
                try:
                    n_months_val = len(hl_monthly_df_clean); ew_pred_mean_val = hl_monthly_df_clean.get('ew_ret_pred_HL', pd.Series(dtype=float)).mean(); vw_pred_mean_val = hl_monthly_df_clean.get('vw_ret_pred_HL', pd.Series(dtype=float)).mean()
                    ew_excess_mean_val = hl_monthly_df_clean.get('ew_ret_excess_HL', pd.Series(dtype=float)).mean(); vw_excess_mean_val = hl_monthly_df_clean.get('vw_ret_excess_HL', pd.Series(dtype=float)).mean()
                    ew_std_val = hl_monthly_df_clean.get('ew_ret_raw_HL', pd.Series(dtype=float)).std(); vw_std_val = hl_monthly_df_clean.get('vw_ret_raw_HL', pd.Series(dtype=float)).std()
                    if pd.notna(ew_std_val) and ew_std_val > 1e-9: ew_sharpe_hl_val = (ew_excess_mean_val / ew_std_val) * np.sqrt(annualization_factor)
                    if pd.notna(vw_std_val) and vw_std_val > 1e-9: vw_sharpe_hl_val = (vw_excess_mean_val / vw_std_val) * np.sqrt(annualization_factor)
                    hl_stats_data = {'ew_pred_mean': [ew_pred_mean_val], 'vw_pred_mean': [vw_pred_mean_val], 'ew_excess_mean': [ew_excess_mean_val], 'vw_excess_mean': [vw_excess_mean_val], 'ew_raw_std': [ew_std_val], 'vw_raw_std': [vw_std_val], 'n_months': [n_months_val], 'ew_sharpe': [ew_sharpe_hl_val], 'vw_sharpe': [vw_sharpe_hl_val], 'Decile': ['H-L']}
                    hl_stats_df = pd.DataFrame(hl_stats_data)
                except Exception as e: print(f"  FEIL H-L calc {model_name}: {e}"); hl_stats_df = pd.DataFrame(); ew_sharpe_hl_val, vw_sharpe_hl_val = np.nan, np.nan
            else: hl_monthly_dfs_plotting.pop(model_name, None); ew_sharpe_hl_val, vw_sharpe_hl_val = np.nan, np.nan
        else: hl_monthly_dfs_plotting.pop(model_name, None); ew_sharpe_hl_val, vw_sharpe_hl_val = np.nan, np.nan
        hl_sharpe_ratios[model_name]['hl_sharpe_ew'] = ew_sharpe_hl_val; hl_sharpe_ratios[model_name]['hl_sharpe_vw'] = vw_sharpe_hl_val
        if not hl_stats_df.empty:
            all_cols = list(set(decile_performance.columns).union(set(hl_stats_df.columns))); decile_performance = decile_performance.reindex(columns=all_cols); hl_stats_df = hl_stats_df.reindex(columns=all_cols)
            model_summary = pd.concat([decile_performance, hl_stats_df], ignore_index=True)
        else: model_summary = decile_performance
        model_summary['Model'] = model_name; performance_summary_list.append(model_summary)

    if not performance_summary_list: print("Ingen ytelsesoppsummeringer generert."); return None, None
    final_summary_df = pd.concat(performance_summary_list).reset_index(drop=True)
    def format_performance_table(df, model_name, weight_scheme):
        model_data = df[df['Model'] == model_name].copy();
        if model_data.empty: return pd.DataFrame()
        cols_map = {'ew_pred_mean': 'Pred', 'ew_excess_mean': 'Real', 'ew_raw_std': 'Std', 'ew_sharpe': 'Sharpe'} if weight_scheme == 'EW' else {'vw_pred_mean': 'Pred', 'vw_excess_mean': 'Real', 'vw_raw_std': 'Std', 'vw_sharpe': 'Sharpe'}
        relevant_cols = [col for col in cols_map.keys() if col in model_data.columns]
        if 'Decile' not in model_data.columns: return pd.DataFrame()
        sub_df = model_data[relevant_cols + ['Decile']].rename(columns=cols_map); sub_df['Decile'] = sub_df['Decile'].astype(str); sub_df = sub_df.set_index('Decile')
        final_cols_display = ['Pred', 'Real', 'Std', 'Sharpe'];
        for col in final_cols_display:
             if col in sub_df.columns: sub_df[col] = pd.to_numeric(sub_df[col], errors='coerce')
        if 'Pred' in sub_df.columns: sub_df['Pred'] = (sub_df['Pred'] * 100).map('{:.2f}%'.format).replace('nan%', 'N/A')
        if 'Real' in sub_df.columns: sub_df['Real'] = (sub_df['Real'] * 100).map('{:.2f}%'.format).replace('nan%', 'N/A')
        if 'Std' in sub_df.columns: sub_df['Std'] = (sub_df['Std'] * 100).map('{:.2f}%'.format).replace('nan%', 'N/A')
        if 'Sharpe' in sub_df.columns: sub_df['Sharpe'] = sub_df['Sharpe'].map('{:.2f}'.format).replace('nan', 'N/A')
        def map_index(x):
            if x == '0.0' or x == '0': return 'Low (L)';
            if x == '9.0' or x == '9': return 'High (H)';
            if x == 'H-L': return 'H-L'
            try: return str(int(float(x)) + 1)
            except ValueError: return x
        sub_df.index = sub_df.index.map(map_index); desired_order = ['Low (L)', '2', '3', '4', '5', '6', '7', '8', '9', 'High (H)', 'H-L']
        sub_df = sub_df.reindex([idx for idx in desired_order if idx in sub_df.index])
        final_cols_present = [col for col in final_cols_display if col in sub_df.columns]
        return sub_df[final_cols_present]

    results_tables = {}
    for model_name in model_names_in_results:
        print(f"\n--- Ytelsestabell: {model_name} - Likevektet (EW) ---"); ew_table = format_performance_table(final_summary_df, model_name, 'EW')
        if not ew_table.empty: print(ew_table); results_tables[f'{model_name}_EW'] = ew_table
        else: print(f"Kunne ikke generere EW-tabell for {model_name}.")
        print(f"\n--- Ytelsestabell: {model_name} - Verdivektet (VW) ---"); vw_table = format_performance_table(final_summary_df, model_name, 'VW')
        if not vw_table.empty: print(vw_table); results_tables[f'{model_name}_VW'] = vw_table
        else: print(f"Kunne ikke generere VW-tabell for {model_name}.")

    plt.figure(figsize=(14, 8)); plot_lines = 0
    sorted_models_for_plot = sorted(hl_monthly_dfs_plotting.keys(), key=lambda x: (x != 'OLS-3+H', x != 'OLS', x.startswith('NN'), x))
    for model_name in sorted_models_for_plot:
        model_hl_df = hl_monthly_dfs_plotting.get(model_name)
        if model_hl_df is not None and 'MonthYear' in model_hl_df.columns:
            if 'ew_ret_raw_HL' in model_hl_df.columns:
                plot_data_ew = model_hl_df.set_index('MonthYear')['ew_ret_raw_HL'].dropna()
                if not plot_data_ew.empty: (1 + plot_data_ew).cumprod().plot(label=f'{model_name} H-L EW'); plot_lines += 1
            if 'vw_ret_raw_HL' in model_hl_df.columns:
                plot_data_vw = model_hl_df.set_index('MonthYear')['vw_ret_raw_HL'].dropna()
                if not plot_data_vw.empty: (1 + plot_data_vw).cumprod().plot(label=f'{model_name} H-L VW', linestyle='--'); plot_lines += 1
    if plot_lines > 0:
        plt.title('Kumulativ RÅ Avkastning for Long-Short Desilporteføljer (Høy - Lav, t+1)'); plt.ylabel('Kumulativ Avkastning (Log-skala)'); plt.xlabel('Måned'); plt.yscale('log')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5)); plt.grid(True, which='both', linestyle='--', linewidth=0.5); plt.tight_layout(rect=[0, 0, 0.85, 1]); plt.show()
    else: print("\nKunne ikke generere plott for kumulativ avkastning.")
    return results_tables, dict(hl_sharpe_ratios)

# Step 6.7: Variable Importance (NN Specific)
def calculate_variable_importance(df_standardized, features, target_col, train_idx, val_idx, nn_config, optimal_lambda1, optimal_lr, epochs=100, batch_size=10000, ensemble_size=10):
    """ Calculates permutation importance for NN using data from the LAST provided indices and optimal hyperparameters. """
    # ... (Function code is identical to previous correct versions) ...
    model_name = nn_config['name']; print(f"\n--- Starter Variabel Viktighet Analyse ({model_name} Ensemble, Siste Vindu Data) ---")
    print(f"    Bruker optimale hyperparametre fra siste vindu: L1={optimal_lambda1:.6f}, LR={optimal_lr:.6f}"); print("    *** ADVARSEL: Denne prosessen (ensemble retraining per feature) er SVÆRT tidkrevende! ***")
    if train_idx is None or val_idx is None or train_idx.empty or val_idx.empty: print("  FEIL: Ugyldige VI indekser."); return pd.DataFrame()
    if pd.isna(optimal_lambda1) or pd.isna(optimal_lr): print("  FEIL: Mangler optimale hyperparametre for VI."); return pd.DataFrame()
    print(f"  Bruker siste Train index ({len(train_idx)} obs), siste Val index ({len(val_idx)} obs)")
    X_train_val = df_standardized.loc[train_idx.union(val_idx), features].values; y_train_val = df_standardized.loc[train_idx.union(val_idx), target_col].values
    valid_y_mask = np.isfinite(y_train_val);
    if X_train_val.shape[0] < 2 or np.nanstd(y_train_val[valid_y_mask]) < 1e-15: print("  FEIL: VI data/varians."); return pd.DataFrame()
    ss_tot = np.sum((y_train_val[valid_y_mask] - y_train_val[valid_y_mask].mean())**2);
    if ss_tot < 1e-15: print("  ADVARSEL: VI SS_tot er nesten null."); return pd.DataFrame()
    input_shape = X_train_val.shape[1]; terminate_nan_vi = callbacks.TerminateOnNaN()
    print(f"  Trener base {model_name} ensemble for VI (på siste Train+Val)..."); base_preds_ensemble = []; base_r2 = np.nan
    try:
        for ens_idx in range(ensemble_size):
            K.clear_session(); tf.random.set_seed(SEED + ens_idx + 3*ensemble_size)
            base_model_vi = build_nn_model(input_shape, nn_config, optimal_lambda1); optimizer_base_vi = Adam(learning_rate=optimal_lr); base_model_vi.compile(optimizer=optimizer_base_vi, loss='mean_squared_error')
            hist_base = base_model_vi.fit(X_train_val, y_train_val, epochs=epochs, batch_size=batch_size, callbacks=[terminate_nan_vi], verbose=0)
            if not np.isnan(hist_base.history['loss']).any(): base_preds_ensemble.append(base_model_vi.predict(X_train_val, batch_size=batch_size).flatten())
            else: base_preds_ensemble = []; break
        if not base_preds_ensemble: raise ValueError(f"Base VI ensemble failed ({model_name}).")
        base_preds_avg = np.mean(np.array(base_preds_ensemble), axis=0); valid_base_pred_mask = np.isfinite(base_preds_avg); combined_valid_mask = valid_y_mask & valid_base_pred_mask
        ss_res_base = np.sum((y_train_val[combined_valid_mask] - base_preds_avg[combined_valid_mask])**2); base_r2 = 1 - (ss_res_base / ss_tot); print(f"  Base R2 ({model_name} ensemble, siste Train+Val): {base_r2:.4f}")
    except Exception as e_base: print(f"  FEIL base VI modell ({model_name}): {e_base}"); return pd.DataFrame()
    if np.isnan(base_r2): print(f"  FEIL: Base R2 for VI er NaN."); return pd.DataFrame()
    print(f"  Beregner {model_name} viktighet (zeroing out & retraining ensemble)..."); importance_results = {}; vi_start_time = time.time()
    for feature_idx, feature_name in enumerate(features):
        print(f"    Permuterer feature: {feature_name} ({feature_idx+1}/{len(features)})...", end=""); feat_time_start = time.time()
        X_train_val_permuted = X_train_val.copy(); X_train_val_permuted[:, feature_idx] = 0; permuted_preds_ensemble = []; permuted_r2 = np.nan
        try:
            for ens_idx in range(ensemble_size):
                K.clear_session(); tf.random.set_seed(SEED + ens_idx + (feature_idx+4)*ensemble_size)
                permuted_model = build_nn_model(input_shape, nn_config, optimal_lambda1); optimizer_perm = Adam(learning_rate=optimal_lr); permuted_model.compile(optimizer=optimizer_perm, loss='mean_squared_error')
                hist_perm = permuted_model.fit(X_train_val_permuted, y_train_val, epochs=epochs, batch_size=batch_size, callbacks=[terminate_nan_vi], verbose=0)
                if not np.isnan(hist_perm.history['loss']).any(): permuted_preds_ensemble.append(permuted_model.predict(X_train_val_permuted, batch_size=batch_size).flatten())
                else: permuted_preds_ensemble = []; break
            if not permuted_preds_ensemble: print(f" -> WARN: Ensemble retraining failed. Assign R2=base."); permuted_r2 = base_r2
            else:
                permuted_preds_avg = np.mean(np.array(permuted_preds_ensemble), axis=0); valid_perm_pred_mask = np.isfinite(permuted_preds_avg); combined_valid_mask_perm = valid_y_mask & valid_perm_pred_mask
                ss_res_permuted = np.sum((y_train_val[combined_valid_mask_perm] - permuted_preds_avg[combined_valid_mask_perm])**2); permuted_r2 = 1 - (ss_res_permuted / ss_tot)
        except Exception as e_perm: print(f" -> FEIL: {e_perm}. Assign R2=base."); permuted_r2 = base_r2
        r2_reduction = base_r2 - permuted_r2; importance_results[feature_name] = max(0, r2_reduction) if pd.notna(r2_reduction) else 0.0
        print(f" [R2 perm: {permuted_r2:.4f}, Red: {importance_results[feature_name]:.4f}, Time: {time.time() - feat_time_start:.1f}s]")
    total_vi_time = time.time() - vi_start_time; print(f"  Total VI beregningstid: {total_vi_time:.1f}s")
    if not importance_results: print("  Ingen VI resultater."); return pd.DataFrame()
    imp_df = pd.DataFrame(importance_results.items(), columns=['Feature', 'R2_reduction']); total_reduction = imp_df['R2_reduction'].sum()
    imp_df['Importance'] = imp_df['R2_reduction']/total_reduction if total_reduction > 1e-9 else 0.0; imp_df = imp_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)
    print(f"\n--- Variabel Viktighet ({model_name} Ensemble, Top 10) ---"); print(imp_df[['Feature', 'Importance']].head(10).round(4))
    plt.figure(figsize=(12, 8)); top_n = 20; plot_df = imp_df.head(top_n).sort_values(by='Importance', ascending=True)
    plt.barh(plot_df['Feature'], plot_df['Importance']); plt.xlabel("Relativ Viktighet (Permutation - Ensemble Retrain)"); plt.ylabel("Feature"); plt.title(f'{model_name} Variabel Viktighet (Top {top_n}) - Siste Vindus Data'); plt.tight_layout(); plt.show()
    return imp_df[['Feature', 'Importance']]

# Step 7: Analyze Prespecified Portfolios (Placeholder)
def analyze_prespecified_portfolios(results_df, original_df_subset, portfolio_definitions_file=None, model_name_label="MODEL"):
    """ Placeholder function for analyzing prespecified portfolios. Needs implementation. """
    # ... (Function code is identical to previous correct versions) ...
    print(f"\n--- Starter Analyse av Prespesifiserte Porteføljer ({model_name_label}, Placeholder) ---")
    pred_col = f'yhat_{model_name_label.lower().replace("-","_").replace("+","h")}' # Adapt if needed
    portfolio_r2_results = pd.DataFrame(); market_timing_results = pd.DataFrame()
    if portfolio_definitions_file is None or not os.path.exists(portfolio_definitions_file): print("  ADVARSEL: Fil med porteføljedefinisjoner mangler. Hopper over."); return portfolio_r2_results, market_timing_results
    print("--- Analyse av Prespesifiserte Porteføljer Fullført (Placeholder) ---")
    return portfolio_r2_results, market_timing_results

# Step 8: Create and Print Summary Table
def create_and_print_summary_table(model_metrics, hl_sharpe_ratios):
    """ Creates and prints a summary table of key metrics for each model. """
    # ... (Function code is identical to previous correct versions) ...
    print("\n\n--- Oppsummerende Resultattabell (Årlig Refitting) ---")
    summary_data = []; models_sorted = sorted(model_metrics.keys(), key=lambda x: (x != 'OLS-3+H', x != 'OLS', x.startswith('NN'), x))
    for model_name in models_sorted:
        metrics = model_metrics[model_name]; hl_sharpes = hl_sharpe_ratios.get(model_name, {'hl_sharpe_ew': np.nan, 'hl_sharpe_vw': np.nan})
        overall_oos_r2 = metrics.get('oos_r2_overall_gu', np.nan); avg_is_r2_gu = np.nanmean(metrics.get('is_r2', [])); avg_oos_sharpe_pred = np.nanmean(metrics.get('oos_sharpe', []))
        summary_data.append({'Modell': model_name, 'IS R² (Avg Gu)': avg_is_r2_gu, 'OOS R² (Overall Gu)': overall_oos_r2, 'OOS Pred. Sharpe (Avg)': avg_oos_sharpe_pred, 'H-L EW Sharpe': hl_sharpes.get('hl_sharpe_ew', np.nan), 'H-L VW Sharpe': hl_sharpes.get('hl_sharpe_vw', np.nan)})
    if not summary_data: print("Ingen data for oppsummeringstabell."); return pd.DataFrame()
    summary_df = pd.DataFrame(summary_data).set_index('Modell')
    summary_df['IS R² (Avg Gu)'] = (summary_df['IS R² (Avg Gu)'] * 100).map('{:.2f}%'.format).replace('nan%', 'N/A')
    summary_df['OOS R² (Overall Gu)'] = (summary_df['OOS R² (Overall Gu)'] * 100).map('{:.2f}%'.format).replace('nan%', 'N/A')
    summary_df['OOS Pred. Sharpe (Avg)'] = summary_df['OOS Pred. Sharpe (Avg)'].map('{:.3f}'.format).replace('nan', 'N/A')
    summary_df['H-L EW Sharpe'] = summary_df['H-L EW Sharpe'].map('{:.3f}'.format).replace('nan', 'N/A')
    summary_df['H-L VW Sharpe'] = summary_df['H-L VW Sharpe'].map('{:.3f}'.format).replace('nan', 'N/A')
    print(summary_df); return summary_df

# Step 9: Main Orchestration Function (Combined Models - Yearly Refitting)
# --- Define Yearly Split Parameters Here ---
INITIAL_TRAIN_YEARS_DEFAULT = 9
VALIDATION_YEARS_DEFAULT = 6
TEST_YEARS_PER_WINDOW_DEFAULT = 1

def run_analysis_for_model_subset(file_path, data_subset, model_config,
                                 # Pass full DataFrame and defined feature lists
                                 df_full, all_numeric_features, ols3_features,
                                 benchmark_file=None, ff_factor_file=None, portfolio_defs_file=None,
                                 filter_portfolio_construction=False,
                                 top_n=1000, bottom_n=1000,
                                 initial_train_years=INITIAL_TRAIN_YEARS_DEFAULT,
                                 val_years=VALIDATION_YEARS_DEFAULT,
                                 test_years=TEST_YEARS_PER_WINDOW_DEFAULT,
                                 # NN Specific Params
                                 nn_param_grid=None, epochs=100, batch_size=10000, patience=5, ensemble_size=10,
                                 # Linear Specific Params
                                 pls_pcr_n_components=10, ridge_alphas=None, lasso_alphas=None, enet_l1_ratio=0.5
                                 ):
    """ Runs the pipeline for ONE model and ONE subset with yearly refitting. """
    model_name = model_config['name']; model_type = model_config['type']; run_label = data_subset.capitalize()
    start_time_model_subset = datetime.datetime.now()
    print(f"\n{'='*20} Starter Kjøring: {model_name} for '{run_label}' Firms (Årlig Refit) {'='*20}")

    # --- Use the pre-processed df for the correct subset ---
    # This requires the main loop to handle subsetting first
    df = df_full # Assume df_full is the already subsetted and cleaned df passed in

    # --- Setup Yearly Rolling Window ---
    print(f"\n--- Steg 4: Setter opp ÅRLIG Rullerende Vindu (InitTrain={initial_train_years}, Val={val_years}, Test={test_years}) ---")
    results_list = []; model_metrics_run = defaultdict(list);
    yhat_col_name = f"yhat_{model_name.lower().replace('+','h').replace('-','')}"
    last_train_idx, last_val_idx = None, None; last_optim_lambda1, last_optim_lr = np.nan, np.nan

    try:
        splits_generator = get_yearly_rolling_splits(df, initial_train_years, val_years, test_years)
        splits = list(splits_generator); num_windows = len(splits)
    except ValueError as e: print(f"FEIL splits ({model_name}/{run_label}): {e}"); return np.nan, None, None, (None,)*6, None, (None, None)
    print(f"Antall årlige rullerende vinduer: {num_windows}\n");
    if num_windows == 0: print(f"Ingen årlige vinduer ({model_name}/{run_label})."); return np.nan, None, None, (None,)*6, None, (None, None)

    # --- Yearly Rolling Window Loop ---
    print(f"--- Starter {model_name} ÅRLIG Rullerende Vindu Trening & Prediksjon ---")
    for window, (train_idx, val_idx, test_idx, train_dates, val_dates, test_dates) in enumerate(splits):
        window_start_time = datetime.datetime.now(); window_num = window + 1; print("-" * 60)
        if test_idx.empty: print(f"Vindu {window_num}: Tomt testsett. Hopper over."); continue
        train_val_idx = train_idx.union(val_idx)
        if len(train_val_idx) < 2 or len(test_idx) < 1: print(f"Vindu {window_num}: Utilstrekkelig data. Hopper over."); continue

        y_train_val = df.loc[train_val_idx, "TargetReturn_t"].values
        y_val_nn = df.loc[val_idx, "TargetReturn_t"].values
        y_test = df.loc[test_idx, "TargetReturn_t"].values
        window_predictions = {'Date': df.loc[test_idx, 'Date'].values, 'Instrument': df.loc[test_idx, 'Instrument'].values, 'TargetReturn_t': y_test}

        X_train_val_formodel = None; X_test_formodel = None; X_val_nn = None
        current_features = model_config['features_list']

        # --- Prepare Features (Apply Scaling ONLY for Linear) ---
        if model_type == 'nn':
            X_train_val_formodel = df.loc[train_val_idx, current_features].values
            X_test_formodel = df.loc[test_idx, current_features].values
            X_val_nn = df.loc[val_idx, current_features].values
            X_train_val_formodel = np.nan_to_num(X_train_val_formodel, nan=0.0, posinf=0.0, neginf=0.0)
            X_test_formodel = np.nan_to_num(X_test_formodel, nan=0.0, posinf=0.0, neginf=0.0)
            X_val_nn = np.nan_to_num(X_val_nn, nan=0.0, posinf=0.0, neginf=0.0)
        else: # Linear models need StandardScaler
            X_train_val_linear = df.loc[train_val_idx, current_features].values
            X_test_linear = df.loc[test_idx, current_features].values
            scaler = StandardScaler()
            try:
                X_train_val_formodel = scaler.fit_transform(X_train_val_linear)
                X_test_formodel = scaler.transform(X_test_linear)
                X_train_val_formodel = np.nan_to_num(X_train_val_formodel, nan=0.0, posinf=0.0, neginf=0.0)
                X_test_formodel = np.nan_to_num(X_test_formodel, nan=0.0, posinf=0.0, neginf=0.0)
            except Exception as e: print(f"  FEIL Scaling {model_name}: {e}. Hopper over vindu."); continue

        # Check for sufficient observations vs features AFTER potential scaling
        if X_train_val_formodel is None: continue # Skip if scaling failed
        min_obs_needed = X_train_val_formodel.shape[1] + 1
        if X_train_val_formodel.shape[0] < min_obs_needed: print(f"     Hopper over {model_name} (for få obs etter scaling: {X_train_val_formodel.shape[0]} < {min_obs_needed})"); continue

        # --- Run the appropriate model ---
        preds_oos = np.full(y_test.shape, np.nan); r2_oos, mse_oos, sharpe_oos, r2_is = (np.nan,) * 4
        optim_lambda1_nn, optim_lr_nn = np.nan, np.nan
        try:
            if model_type == 'nn':
                 if nn_param_grid is None: raise ValueError("NN param grid missing")
                 current_batch_size = min(batch_size, len(train_idx)) if len(train_idx) > 0 else 1
                 if current_batch_size != batch_size and len(train_idx) > 0: print(f"  INFO ({model_name}): Batch size justert til {current_batch_size}")
                 # Separate X_train for initial NN fit during tuning
                 X_train_nn = df.loc[train_idx, current_features].values
                 X_train_nn = np.nan_to_num(X_train_nn, nan=0.0, posinf=0.0, neginf=0.0)
                 y_train_nn = df.loc[train_idx, "TargetReturn_t"].values

                 preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is, optim_lambda1_nn, optim_lr_nn = run_nn_on_window(
                     X_train_nn, y_train_nn, X_val_nn, y_val_nn, X_test_formodel, y_test, # Pass correct arrays
                     nn_config=model_config, param_grid=list(ParameterGrid(nn_param_grid)),
                     epochs=epochs, batch_size=current_batch_size, patience=patience, ensemble_size=ensemble_size
                 )
            elif model_type == 'statsmodels':
                 preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is, _, _ = run_statsmodels_on_window(X_train_val_formodel, y_train_val, X_test_formodel, y_test, method=model_name)
            elif model_type == 'sklearn':
                 model_params = model_config.get('params', {})
                 # Add specific params if needed
                 if model_name in ['PLS', 'PCR']: model_params['n_components'] = pls_pcr_n_components
                 if model_name == 'RIDGE': model_params['alphas_ridge'] = ridge_alphas
                 if model_name in ['LASSO', 'ELASTICNET']: model_params['alphas_lasso'] = lasso_alphas
                 if model_name == 'ELASTICNET': model_params['l1_ratio_elastic'] = enet_l1_ratio
                 preds_oos, r2_oos, mse_oos, sharpe_oos, r2_is, _, _ = run_sklearn_on_window(X_train_val_formodel, y_train_val, X_test_formodel, y_test, model_type=model_name, **model_params)
        except Exception as e: print(f"  FEIL under kjøring av {model_name}: {e}") #; traceback.print_exc()

        # Store results
        window_predictions[yhat_col_name] = preds_oos; model_metrics_run['oos_r2'].append(r2_oos); model_metrics_run['oos_sharpe'].append(sharpe_oos); model_metrics_run['is_r2'].append(r2_is)
        if model_type == 'nn': model_metrics_run['optim_lambda1'].append(optim_lambda1_nn); model_metrics_run['optim_lr'].append(optim_lr_nn)
        if model_type == 'nn' and pd.notna(optim_lambda1_nn) and pd.notna(optim_lr_nn): last_train_idx = train_idx.copy(); last_val_idx = val_idx.copy(); last_optim_lambda1 = optim_lambda1_nn; last_optim_lr = optim_lr_nn

        print(f"  Vindu {window_num} ({model_name}) fullført på {(datetime.datetime.now() - window_start_time).total_seconds():.1f} sek. OOS R2: {r2_oos:.4f}, IS R2: {r2_is:.4f}")
        if model_type == 'nn': K.clear_session()

    # --- Aggregate Results & Overall Analysis ---
    if not results_list: print(f"\nFEIL: Ingen OOS resultater for {model_name}/{run_label}."); return np.nan, None, None, (None,)*6, None, (None, None)
    results_df = pd.concat(results_list).reset_index(drop=True)
    print(f"\n--- Samlet Resultatanalyse ({model_name}, {run_label}) ---")
    y_true_all=results_df['TargetReturn_t']; y_pred_all=results_df[yhat_col_name]; valid_idx_all = y_true_all.notna() & y_pred_all.notna() & np.isfinite(y_true_all) & np.isfinite(y_pred_all)
    y_t_valid_all=y_true_all[valid_idx_all]; y_p_valid_all=y_pred_all[valid_idx_all]; R2OOS_overall=np.nan
    if len(y_t_valid_all)>1: ss_res_all=np.sum((y_t_valid_all-y_p_valid_all)**2); ss_true_sq_all=np.sum(y_t_valid_all**2);
    if ss_true_sq_all > 1e-15: R2OOS_overall = 1 - (ss_res_all / ss_true_sq_all)
    avg_yearly_window_r2 = np.nanmean(model_metrics_run['oos_r2'])
    print(f"Overall OOS R² ({model_name}, {run_label}, Gu): {R2OOS_overall:.6f}"); print(f"Average Yearly Window OOS R² ({model_name}, {run_label}):  {avg_yearly_window_r2:.6f}"); model_metrics_run['oos_r2_overall_gu'] = R2OOS_overall

    # --- Detailed Portfolio Analysis ---
    # Use the original FULL df (before yearly splits) for this, merged with the aggregated results_df
    portfolio_tables, hl_sharpe_ratios_run = calculate_portfolio_performance(results_df, df, prediction_cols=[yhat_col_name], benchmark_file=benchmark_file, ff_factor_file=ff_factor_file, filter_small_caps=filter_portfolio_construction, model_name_label=model_name)

    # --- Analyze Prespecified Portfolios (Placeholder) ---
    prespec_r2_table, prespec_timing_table = analyze_prespecified_portfolios(results_df, df, portfolio_definitions_file=portfolio_defs_file, model_name_label=model_name)

    # --- Variable Importance (Only for NNs on 'all' subset) ---
    vi_df = None
    if model_type == 'nn' and data_subset == 'all':
        print(f"\n--- Forbereder Variabel Viktighet Analyse ({model_name}) ---")
        if last_train_idx is not None and last_val_idx is not None and pd.notna(last_optim_lambda1) and pd.notna(last_optim_lr):
             # Pass the original df which contains the rank standardized features
             vi_df = calculate_variable_importance(df, current_features, 'TargetReturn_t', last_train_idx, last_val_idx, model_config, last_optim_lambda1, last_optim_lr, epochs, batch_size, ensemble_size)
        else: print("  Kan ikke kjøre VI: mangler siste vindus info.")
    elif model_type != 'nn': print(f"\n--- Hopper over Variabel Viktighet (ikke NN-modell) ---")

    # --- Final Summary ---
    end_time = datetime.datetime.now(); print(f"\n--- Kjøring ({run_label}, {model_name}) fullført: {(end_time - start_time_model_subset)} ---")
    # **MODIFIED RETURN**: Include hl_sharpe_ratios_run
    return R2OOS_overall, vi_df, dict(model_metrics_run), portfolio_tables, hl_sharpe_ratios_run, (prespec_r2_table, prespec_timing_table)

# --------------------------------------------------------------------------
# Main Execution Block (Combined Models - Yearly Refitting)
# --------------------------------------------------------------------------
if __name__ == "__main__":
    # --- General Configuration ---
    DATA_FILE = "Cleaned_OSEFX_Market_Macro_Data.csv"
    BENCHMARK_CSV_FILE = None
    FF_FACTOR_CSV_FILE = None
    PORTFOLIO_DEFS_CSV_FILE = None
    OUTPUT_BASE_DIR = "Combined_Results_YearlyRefit"
    FILTER_SMALL_CAPS_PORTFOLIO = False
    TOP_N_FIRMS = 1000; BOTTOM_N_FIRMS = 1000

    # --- Yearly Split Parameters ---
    INIT_TRAIN_YEARS = 9; VAL_YEARS = 6; TEST_YEARS = 1

    # --- NN Specific Hyperparameters ---
    NN_PARAM_GRID = {'lambda1': [1e-5, 1e-4, 1e-3], 'learning_rate': [0.001, 0.01]}
    NN_EPOCHS = 100; NN_BATCH_SIZE = 10000; NN_PATIENCE = 5; NN_ENSEMBLE_SIZE = 10

    # --- Linear Model Specific Parameters ---
    PLS_PCR_N_COMPONENTS = 10; RIDGE_ALPHAS = np.logspace(-4, 4, 10); LASSO_ALPHAS = np.logspace(-6, -1, 10); ENET_L1_RATIO = 0.5

    # --- Load Data ONCE and Define Features ONCE ---
    print("Laster data og definerer features en gang...")
    df_initial = load_prepare_data(DATA_FILE)
    if df_initial is None: exit()
    ALL_NUMERIC_FEATURES, OLS3_FEATURES = define_features(df_initial)
    if not ALL_NUMERIC_FEATURES: print("FEIL: Ingen numeriske features. Avslutter."); exit()
    print("-" * 30)

    # --- Define ALL Models To Run ---
    ALL_MODELS_CONFIG = {}
    if OLS3_FEATURES: ALL_MODELS_CONFIG['OLS-3+H'] = {'name': 'OLS-3+H', 'type': 'statsmodels', 'features_list': OLS3_FEATURES}
    if ALL_NUMERIC_FEATURES:
        ALL_MODELS_CONFIG['OLS'] = {'name': 'OLS', 'type': 'sklearn', 'features_list': ALL_NUMERIC_FEATURES}
        ALL_MODELS_CONFIG['PLS'] = {'name': 'PLS', 'type': 'sklearn', 'features_list': ALL_NUMERIC_FEATURES, 'params': {'n_components': PLS_PCR_N_COMPONENTS}}
        ALL_MODELS_CONFIG['PCR'] = {'name': 'PCR', 'type': 'sklearn', 'features_list': ALL_NUMERIC_FEATURES, 'params': {'n_components': PLS_PCR_N_COMPONENTS}}
        ALL_MODELS_CONFIG['RIDGE'] = {'name': 'RIDGE', 'type': 'sklearn', 'features_list': ALL_NUMERIC_FEATURES, 'params': {'alphas_ridge': RIDGE_ALPHAS}}
        ALL_MODELS_CONFIG['LASSO'] = {'name': 'LASSO', 'type': 'sklearn', 'features_list': ALL_NUMERIC_FEATURES, 'params': {'alphas_lasso': LASSO_ALPHAS}}
        ALL_MODELS_CONFIG['ELASTICNET'] = {'name': 'ELASTICNET', 'type': 'sklearn', 'features_list': ALL_NUMERIC_FEATURES, 'params': {'alphas_lasso': LASSO_ALPHAS, 'l1_ratio_elastic': ENET_L1_RATIO}}
    nn_archs = {'NN1': [32], 'NN2': [64, 32], 'NN3': [96, 64, 32], 'NN4': [128, 96, 64, 32], 'NN5': [128, 96, 64, 32, 16]}
    if ALL_NUMERIC_FEATURES: # Use ALL features for NNs after rank standardization
        for name, units in nn_archs.items(): ALL_MODELS_CONFIG[name] = {'name': name, 'type': 'nn', 'features_list': ALL_NUMERIC_FEATURES, 'hidden_units': units}

    # --- Check Files & Create Output Dir ---
    if not os.path.exists(OUTPUT_BASE_DIR): os.makedirs(OUTPUT_BASE_DIR); print(f"Opprettet base mappe: {OUTPUT_BASE_DIR}")
    # ... (add checks for benchmark/factor/portfolio files if paths are set) ...

    # --- Run Analysis for Each Model and Subset ---
    all_results_r2_summary = defaultdict(dict)
    all_variable_importance = {}
    all_model_metrics_agg = defaultdict(dict)
    all_portfolio_tables_agg = defaultdict(lambda: defaultdict(dict)) # Store tables[subset][model]
    all_prespecified_results_agg = defaultdict(dict)
    final_hl_sharpe_ratios = {} # Store HL Sharpe from 'all' subset runs

    # --- Loop through each SUBSET ---
    for subset in ['all', 'big', 'small']:
        print(f"\n{'#'*30} Processing Subset: {subset.upper()} {'#'*30}")

        # --- Create Subset Data ---
        print(f"--- Steg 1.1: Lager Subset: {subset.capitalize()} ---")
        df_subset = pd.DataFrame()
        df_raw_mc = df_initial.dropna(subset=['MarketCap_orig', 'Date']) # Use initially loaded df
        if subset == 'all': df_subset = df_initial.copy()
        elif subset == 'big': df_raw_mc['MonthYear'] = df_raw_mc['Date'].dt.to_period('M'); df_subset = df_raw_mc.groupby('MonthYear', group_keys=False).apply(lambda x: x.nlargest(TOP_N_FIRMS, "MarketCap_orig")); df_subset = df_subset.drop(columns=['MonthYear'])
        elif subset == 'small': df_raw_mc['MonthYear'] = df_raw_mc['Date'].dt.to_period('M'); df_subset = df_raw_mc.groupby('MonthYear', group_keys=False).apply(lambda x: x.nsmallest(BOTTOM_N_FIRMS, "MarketCap_orig")); df_subset = df_subset.drop(columns=['MonthYear'])
        if df_subset.empty: print(f"FEIL: Subset '{subset}' er tomt. Hopper over."); continue
        print(f"Subset '{subset}' initiell form: {df_subset.shape}")

        # --- Apply Rank Standardization and Cleaning to the Subset ---
        print(f"\n--- Steg 1.5 & 3: Rank Standardiserer og Renser Subset: {subset.capitalize()} ---")
        # Apply Rank Std to ALL features *before* cleaning (needed for NNs)
        df_subset_ranked = rank_standardize_features(df_subset, ALL_NUMERIC_FEATURES)
        essential_cols_list_sub = list(set(ALL_NUMERIC_FEATURES + OLS3_FEATURES + ['TargetReturn_t', 'NextMonthlyReturn_t+1', 'MarketCap_orig', 'Date', 'Instrument']))
        essential_cols_for_dropna_sub = [col for col in essential_cols_list_sub if col in df_subset_ranked.columns]
        df_subset_cleaned = clean_data(df_subset_ranked, ALL_NUMERIC_FEATURES, essential_cols_for_dropna_sub, target="TargetReturn_t")
        if df_subset_cleaned.empty: print(f"FEIL: Subset '{subset}' tomt etter rensing. Hopper over."); continue
        # Refresh feature lists based on cleaned data for this subset
        ols3_features_sub = [f for f in OLS3_FEATURES if f in df_subset_cleaned.columns]
        all_numeric_features_sub = [f for f in ALL_NUMERIC_FEATURES if f in df_subset_cleaned.columns and df_subset_cleaned[f].nunique()>1 and df_subset_cleaned[f].std()>1e-9]
        if not all_numeric_features_sub: print(f"FEIL: Ingen features igjen for subset {subset}. Hopper over."); continue


        # --- Loop through each MODEL for this subset ---
        for model_key, model_conf in ALL_MODELS_CONFIG.items():
            model_name = model_conf['name']
            model_output_dir = os.path.join(OUTPUT_BASE_DIR, model_name) # Save results per model

            # Update model config with subset-specific feature lists
            current_conf = model_conf.copy()
            if model_name == 'OLS-3+H':
                if not ols3_features_sub: print(f"INFO: Hopper over {model_name} for subset {subset} (OLS3 features mangler)."); continue
                current_conf['features_list'] = ols3_features_sub
            else: # All other models use all available numeric features
                if not all_numeric_features_sub: print(f"INFO: Hopper over {model_name} for subset {subset} (ingen numeriske features)."); continue
                current_conf['features_list'] = all_numeric_features_sub


            # Run analysis for this specific model and subset
            # Pass the cleaned subset DataFrame
            r2, vi_df_run, metrics_run, port_tables_run, hl_sharpes_run, prespec_tables_run = run_analysis_for_model_subset(
                file_path=DATA_FILE, # File path still needed by load_prepare_data if called inside
                data_subset=subset,
                model_config=current_conf, # Use the updated config
                df_full=df_subset_cleaned, # Pass the cleaned subset data
                all_numeric_features=all_numeric_features_sub, # Pass subset features
                ols3_features=ols3_features_sub,
                benchmark_file=BENCHMARK_CSV_FILE, ff_factor_file=FF_FACTOR_CSV_FILE, portfolio_defs_file=PORTFOLIO_DEFS_CSV_FILE,
                filter_portfolio_construction=FILTER_SMALL_CAPS_PORTFOLIO, top_n=TOP_N_FIRMS, bottom_n=BOTTOM_N_FIRMS,
                initial_train_years=INIT_TRAIN_YEARS, val_years=VAL_YEARS, test_years=TEST_YEARS,
                nn_param_grid=NN_PARAM_GRID, epochs=NN_EPOCHS, batch_size=NN_BATCH_SIZE, patience=NN_PATIENCE, ensemble_size=NN_ENSEMBLE_SIZE,
                pls_pcr_n_components=PLS_PCR_N_COMPONENTS, ridge_alphas=RIDGE_ALPHAS, lasso_alphas=LASSO_ALPHAS, enet_l1_ratio=ENET_L1_RATIO
            )

            # Store aggregated results
            all_results_r2_summary[subset][model_name] = r2
            all_model_metrics_agg[subset][model_name] = metrics_run
            all_portfolio_tables_agg[subset][model_name] = port_tables_run # Store tuple of 6 tables
            all_prespecified_results_agg[subset][model_name] = prespec_tables_run
            if subset == 'all' and vi_df_run is not None: all_variable_importance[model_name] = vi_df_run
            # **Store HL Sharpe Ratios from the 'all' subset run**
            if subset == 'all' and hl_sharpes_run:
                 final_hl_sharpe_ratios[model_name] = hl_sharpes_run.get(model_name, {'hl_sharpe_ew': np.nan, 'hl_sharpe_vw': np.nan})

            # --- Save portfolio tables for this subset/model run ---
            if port_tables_run and len(port_tables_run) == 6:
                table_names = ['decile_ew', 'decile_vw', 'hl_risk_ew', 'hl_risk_vw', 'long_risk_ew', 'long_risk_vw']
                for i, table_df in enumerate(port_tables_run):
                    if table_df is not None and not table_df.empty:
                        filename = os.path.join(model_output_dir, f"portfolio_{subset}_{table_names[i]}_yearly.csv")
                        try: table_df.to_csv(filename); print(f" -> Porteføljetabell lagret: {filename}")
                        except Exception as e: print(f"  FEIL lagring porteføljetabell {filename}: {e}")

            # --- Save prespecified portfolio tables ---
            if prespec_tables_run and len(prespec_tables_run) == 2:
                 names_prespec = ['R2oos_prespecified', 'TimingGains_prespecified']
                 for i, table_df in enumerate(prespec_tables_run):
                      if table_df is not None and not table_df.empty:
                           filename = os.path.join(model_output_dir, f"prespecified_{subset}_{names_prespec[i]}_yearly.csv")
                           try: table_df.to_csv(filename); print(f" -> Prespesifisert tabell lagret: {filename}")
                           except Exception as e: print(f"  FEIL lagring prespesifisert tabell {filename}: {e}")

        # --- Save VI results for this NN model (if calculated) ---
        if model_name in all_variable_importance:
             print(f"\nLagrer Variabel Viktighet resultater for {model_name}...")
             try:
                  vi_filename = os.path.join(model_output_dir, "variable_importance_last_window_yearly.csv")
                  all_variable_importance[model_name].to_csv(vi_filename, index=False)
                  print(f" -> Variabel Viktighet ({model_name}) lagret til {vi_filename}")
             except Exception as e: print(f"  FEIL lagring VI for {model_name}: {e}")

    # --- Aggregate Metrics Across All Models for Final Summary ---
    final_aggregated_metrics = defaultdict(lambda: defaultdict(list))
    for subset, models in all_model_metrics_agg.items():
        for model_name, metrics in models.items():
             final_aggregated_metrics[model_name]['is_r2'].extend(metrics.get('is_r2', []))
             final_aggregated_metrics[model_name]['oos_sharpe'].extend(metrics.get('oos_sharpe', []))
             if 'oos_r2_overall_gu' in metrics:
                 final_aggregated_metrics[model_name]['oos_r2_overall_gu'] = metrics['oos_r2_overall_gu']

    # --- Print Final Consolidated R2 Summary Table ---
    print("\n\n" + "="*30 + " Final Consolidated OOS R2 Summary (All Models - Yearly Refit) " + "="*30)
    summary_r2_for_df = defaultdict(dict)
    for subset, model_results in all_results_r2_summary.items():
        for model_name_key, r2_val in model_results.items():
            summary_r2_for_df[subset][model_name_key] = r2_val
    r2_summary_df = pd.DataFrame.from_dict(summary_r2_for_df, orient='index')
    r2_summary_df.index.name = "Subset"
    model_order = ['OLS-3+H', 'OLS', 'PLS', 'PCR', 'RIDGE', 'LASSO', 'ELASTICNET'] + [f'NN{i}' for i in range(1, 6)]
    cols_ordered = [m for m in model_order if m in r2_summary_df.columns]
    r2_summary_df = r2_summary_df[cols_ordered]
    r2_summary_df_pct = r2_summary_df * 100
    print(r2_summary_df_pct.round(4))
    try:
        r2_summary_filename = os.path.join(OUTPUT_BASE_DIR, "combined_consolidated_r2_summary_yearly.csv")
        r2_summary_df.to_csv(r2_summary_filename)
        print(f"\n -> Konsolidert R2 Sammendrag lagret til {r2_summary_filename}")
    except Exception as e: print(f"  FEIL lagring R2 Sammendrag: {e}")
    print("="*78)

    # --- Print Combined Summary Table (Using aggregated metrics and final HL sharpe) ---
    final_summary_table = create_and_print_summary_table(final_aggregated_metrics, final_hl_sharpe_ratios)
    try:
        if final_summary_table is not None and not final_summary_table.empty:
             summary_filename_all = os.path.join(OUTPUT_BASE_DIR, "combined_summary_metrics_yearly.csv")
             final_summary_table.to_csv(summary_filename_all)
             print(f" -> Kombinert Oppsummeringstabell lagret til {summary_filename_all}")
    except Exception as e: print(f"  FEIL lagring Kombinert Oppsummering: {e}")


    print("\nFull kombinert analyse (Lineær + NN - Årlig Refitting) fullført.")